# DATA

In [1]:
import imgaug as ia
import imgaug.augmenters as iaa
import imageio
import json
import os
import cv2
import numpy as np
import os 
import json
import matplotlib.pylab as plt
from PIL import Image
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from os import listdir
from glob import glob

p = os.getcwd() 

In [2]:
#회전
def rotation(img_path,txt_path, image, label):
    ia.seed(1)
    with open(label) as f:
        json_data = json.load(f)
    image = imageio.imread(image)
    # file_name = json_data['description']['image'].split('.')[0]
    
    xtl = json_data['annotations']['points'][0]['xtl']
    ytl = json_data['annotations']['points'][0]['ytl']
    xbr = json_data['annotations']['points'][0]['xbr']
    ybr = json_data['annotations']['points'][0]['ybr']
    w = json_data['description']['width']
    h = json_data['description']['height']
    
    area = json_data['annotations']['area']
    name = json_data['annotations']['disease']
    risk = json_data['annotations']['risk']
    grow = json_data['annotations']['grow']

    if name == 0:
        if grow == 11:
            c = 0
        else:
            c = 1
    elif name == 5:
        c = risk + 1
    elif name == 6:
        c = 4 + risk

    bf = []
    bf.append([int(xtl), int(ytl), int(xbr), int(ybr)])
    ia_bf = []
    for box in bf:
        ia_bf.append(ia.BoundingBox(x1=box[0],y1=box[1],x2=box[2],y2=box[3],label = c))
        
    bbs = ia.BoundingBoxesOnImage(ia_bf, shape = image.shape)
    name2 = label.split('\\')[-1]
    name_re = name2.split('.')[0]
    print('c = '+str(c))
    for j in range(1, 4):
        rtt = 90*j
        seq = iaa.Sequential([
            iaa.Affine(rotate=rtt)
        ])
        img_aug, bbs_aug = seq(image = image,bounding_boxes = bbs)
        
        x1 = bbs_aug.bounding_boxes[0].x1
        y1 = bbs_aug.bounding_boxes[0].y1
        x2 = bbs_aug.bounding_boxes[0].x2
        y2 = bbs_aug.bounding_boxes[0].y2
            
        w = bbs_aug.shape[1]
        h = bbs_aug.shape[0]
        # print(name2)
        # print(bbs_aug)
        # print(j)
        if x1 < 0:
            x1 = 0
        else : pass
        if y1 < 0:
            y1 = 0
        else : pass
        if x2 < 0:
            x2 = 0
        else : pass
        if y2 < 0:
            y2 = 0  
        else : pass
        x_center = (x1 + (x2-x1)/2)/w
        y_center = (y1 + (y2-y1)/2)/h
        x = (x2-x1)/w
        y = (y2-y1)/h
        # # image_after = bbs_aug.draw_on_image(img_aug, thickness=10, color = [0, 0, 255])
        cv2.imwrite(img_path+str(j)+'-'+name_re+'.jpg',img_aug)
        fw = open(txt_path+str(j)+'-'+name_re+".txt", 'w')
        fw.write(str(c)+' '+str(x_center)+' '+str(y_center)+' '+str(x)+' '+str(y))
        f.close()
            
        # # cv2_imshow(cv2.resize(image_after, (w,h)))
        print('c = '+str(c))

In [3]:
#json -> yolo txt
def convert(path, label):
    with open(label) as f:
        json_data = json.load(f)
    if json_data['annotations']['area'] == 3:
        file_name = json_data['description']['image'].split('.')[0]
        xtl = json_data['annotations']['points'][0]['xtl']
        ytl = json_data['annotations']['points'][0]['ytl']
        xbr = json_data['annotations']['points'][0]['xbr']
        ybr = json_data['annotations']['points'][0]['ybr']

        w = json_data['description']['width']
        h = json_data['description']['height']

        x_center = (xtl + (xbr-xtl)/2)/w
        y_center = (ytl + (ybr-ytl)/2)/h
        x = (xbr-xtl)/w
        y = (ybr-ytl)/h

        area = json_data['annotations']['area']
        name = json_data['annotations']['disease']
        risk = json_data['annotations']['risk']
        grow = json_data['annotations']['grow']


        if name == 0:
            if grow == 11:
                c = 0
            else:
                c = 1
        elif name == 5:
            c = risk + 1
        elif name == 6:
            c = 4 + risk
        fw = open(path+file_name+".txt", 'w')
        fw.write(str(c)+' '+str(x_center)+' '+str(y_center)+' '+str(x)+' '+str(y)+'\n')
        f.close()
        # print(file_name)
    else:
        os.remove(label)


In [4]:
#분포
def distribution(label,lst):
    with open(label,'r') as f:
        read = f.read(1)
        lst.append(read)
    return lst

def distribution2(lst,dic):
    for i in lst:
        try: dic[i] += 1
        except: dic[i] = 1
    fa = sorted(dic.items())
    return fa

In [6]:
#그래프 표시
def grape(x,y):
    plt.bar(x,y)
    for k, v in enumerate(x):
        plt.text(v, y[k], y[k],                 
             fontsize = 9, 
             color='red',
             horizontalalignment='center',  
             verticalalignment='bottom')    # verticalalignment (top, center, bottom)
    plt.show


In [7]:
#데이터 지우기
def Data_remove(img, txt, dic, cnt):
    i = 0
    for file in txt:
        with open(txt[i],'r') as f:
            read = f.read(1)
        if dic[read] == int(cnt):
            i = i + 1
            pass
        else:
            print(read)
            print(img[i])
            print(txt[i])
            os.remove(img[i])
            os.remove(txt[i])
            dic[read] = dic[read] - 1
            print(dic)
            i = i + 1

In [27]:
#밝기
def color_brightness(img_path, txt_path, name, img, label, value):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
    h, s, v = cv2.split(hsv) 
    
    lim = 255 - value 
    v[v > lim] = 255 
    v[v <= lim] += value 
    
    final_hsv = cv2.merge((h, s, v)) 
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)


    cv2.imwrite(img_path+'c'+name+'.jpg', img)

    bf = open(label,'r')
    af = open(txt_path+'c'+name+".txt", 'w')
    bl = bf.readlines()
    for inStr in bl:
        af.writelines(inStr)
    
    bf.close()
    af.close()

In [36]:
#블러
def blurs(img_path, txt_path, name, img, label):
    blur1 = cv2.GaussianBlur(img,(5,5),0)
    cv2.imwrite(img_path+'g'+name+'.jpg', blur1)

    bf = open(label,'r')
    af = open(txt_path+'g'+name+".txt", 'w')
    bl = bf.readlines()
    for inStr in bl:
        af.writelines(inStr)
    
    bf.close()
    af.close()
    
    blur2 = cv2.medianBlur(img,5)
    cv2.imwrite(img_path+'m'+name+'.jpg', blur2)

    bf = open(label,'r')
    af = open(txt_path+'m'+name+".txt", 'w')
    bl = bf.readlines()
    for inStr in bl:
        af.writelines(inStr)
    
    bf.close()
    af.close()

In [9]:
import os 
t = os.getcwd() +'\datasets\\train\\labels\\'
v = os.getcwd() +'\datasets\\valid\\labels\\'
train_label_json = glob(p + '\\label\\label-t\\*')
valid_label_json = glob(p + '\\label\\label-v\\*')
for i in range(len(train_label_json)):
    convert(t,train_label_json[i])

for i in range(len(valid_label_json)):
    convert(v,valid_label_json[i])
    
train_label_txt = glob(p+'/datasets/train/labels/*')
valid_label_txt = glob(p+'/datasets/valid/labels/*')
print(len(train_label_txt), len(valid_label_txt))

800 40


In [29]:
#데이터 / 데이터 정렬
t = os.getcwd() +'\datasets\\train\\labels\\'
v = os.getcwd() +'\datasets\\valid\\labels\\'
train_label_json = glob(p + '\\label\\label-t\\*')
valid_label_json = glob(p + '\\label\\label-v\\*')
train_label_txt = glob(p+'\\datasets\\train\\labels\\*')
valid_label_txt = glob(p+'\\datasets\\valid\\labels\\*')
train_img = glob(p+'\\datasets\\train\\images\\*')
valid_img = glob(p+'\\datasets\\valid\\images\\*')

train_img.sort()
valid_img.sort()
train_label_json.sort()
valid_label_json.sort()
train_label_txt.sort()
valid_label_txt.sort()

print(len(train_img),len(valid_img),len(train_label_json),len(valid_label_json),len(train_label_txt), len(valid_label_txt))


800 40 800 40 800 40


In [15]:
img_train_path = glob('C:\\Users\\a\\Desktop\\swproject\\test\\train\\images\\*')
txt_train_path = glob('C:\\Users\\a\\Desktop\\swproject\\test\\train\\labels\\*')


img_valid_path = glob('C:\\Users\\a\\Desktop\\swproject\\test\\valid\\images\\*')
txt_valid_path = glob('C:\\Users\\a\\Desktop\\swproject\\test\\valid\\labels\\*')

print(len(img_train_path), len(img_valid_path))

2400 120


In [22]:
img_path_t = 'C:\\Users\\a\\Desktop\\swproject\\test\\train\\images\\'
txt_path_t = 'C:\\Users\\a\\Desktop\\swproject\\test\\train\\labels\\'

img_path_v = 'C:\\Users\\a\\Desktop\\swproject\\test\\valid\\images\\'
txt_path_v = 'C:\\Users\\a\\Desktop\\swproject\\test\\valid\\labels\\'



for i in range(len(train_img)):
    rotation(img_path_v, txt_path_v ,valid_img[i],valid_label_json[i])


In [30]:
for i in range(len(train_img)):
    sp = train_img[i].split('\\')[-1]
    name = sp.split('.')[0]
    sample = cv2.imread(train_img[i])
    color_brightness(img_path_t, txt_path_t, name, sample, train_label_txt[i], 50)

for i in range(len(valid_img)):
    sp = valid_img[i].split('\\')[-1]
    name = sp.split('.')[0]
    sample = cv2.imread(valid_img[i])
    color_brightness(img_path_v, txt_path_v, name, sample, valid_label_txt[i], 50)

In [37]:
for i in range(len(train_img)):
    sp = train_img[i].split('\\')[-1]
    name = sp.split('.')[0]
    sample = cv2.imread(train_img[i])
    blurs(img_path_t, txt_path_t, name, sample, train_label_txt[i])

for i in range(len(valid_img)):
    sp = valid_img[i].split('\\')[-1]
    name = sp.split('.')[0]
    sample = cv2.imread(valid_img[i])
    blurs(img_path_v, txt_path_v, name, sample, valid_label_txt[i])

In [ ]:
d_train = []
d_train2 = {}

for i in range(len(train_label_txt)):
    distribution(train_label_txt[i],d_train)

trainx, trainy = zip(*distribution2(d_train,d_train2))

grape(trainx, trainy)


In [ ]:
d_valid = []
d_valid2 = {}

for i in range(len(valid_label_txt)):
    distribution(valid_label_txt[i],d_valid)

validx, validy = zip(*distribution2(d_valid,d_valid2))

grape(validx, validy)


In [38]:
test = glob(p+'\\test\\train\\images\\*')
print(len(test))

4800


In [ ]:
Data_remove(train_label_json,train_label_txt,d_train2,100)
Data_remove(valid_label_json,valid_label_txt,d_valid2,5)


# YOLOV5

In [ ]:
#git yolo / requirments
p = os.getcwd() 

%cd $p
!git clone https://github.com/ultralytics/yolov5

%cd yolov5
!pip install -r requirements.txt

In [4]:
#yaml edit
import yaml

with open(p+'/datasets/data.yaml','r') as f:
    data = yaml.safe_load(f)

data['train'] = p + '\\datasets\\train\\images\\'
data['val'] = p + '\\datasets\\valid\\images\\'
data['nc'] = 8
data['names'] = ['normal','normal+', 'risk-1-1', 'risk-1-2', 'risk-1-3', 'risk-2-1', 'risk-2-2', 'risk-2-3']

with open(p+'/datasets/data.yaml','w') as f:
    yaml.dump(data,f)

In [5]:
#train yolo
!python train.py --img 640 --batch 80 --epochs 300 --data $p/datasets/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name s  --cache

^C


In [9]:
!echo "# hyun3167" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/Nbbang-Computer-science-team-project/hyun3167.git
!git push -u origin main

Reinitialized existing Git repository in C:/Users/oo/Desktop/swproject/yolov5/.git/


Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'oo@DESKTOP-GQ6JF6T.(none)')
error: remote origin already exists.


^C


fatal: helper error (-1): User cancelled dialog.
bash: /dev/tty: No such device or address
error: failed to execute prompt script (exit code 1)
fatal: could not read Username for 'https://github.com': No such file or directory
